# Statistics on the dataset

In [2]:
#import drill package and add it to the jupyter notebook environment
import sys
sys.path.append('../')
import dill

In [15]:
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

source of the dataset

author = {Feng, Tao and Yue, Chuan},
title = {Visualizing and Interpreting RNN Models in URL-Based Phishing Detection},
year = {2020},
isbn = {9781450375689},
publisher = {Association for Computing Machinery},
address = {New York, NY, USA},
url = {https://doi.org/10.1145/3381991.3395602},
doi = {10.1145/3381991.3395602},
booktitle = {Proceedings of the 25th ACM Symposium on Access Control Models and Technologies},
pages = {13–24},
numpages = {12},
keywords = {visualization, interpretation, phishing detection, deep learning},
location = {Barcelona, Spain},
series = {SACMAT '20}
}```

Data import into a global dataframe

⚠ The data that we import is already encoded into numbers for their RNN analysis

In [4]:
import dill
dill_file="vonDataset20180426.dill"
with open(dill_file, 'rb') as f:
        pickleData=dill.load( f )
        train_x,train_y=pickleData["train_x"],pickleData["train_y"]
        val_x,val_y=pickleData["val_x"],pickleData["val_y"]
        test_x,test_y=pickleData["test_x"],pickleData["test_y"]
        char_to_int=pickleData["char_to_int"]
print("Feature Shapes:\n")
print("Train set: {}".format(train_x.shape), 
      " Validation set: {}".format(val_x.shape),
      " Test set: {}".format(test_x.shape))

Feature Shapes:

Train set: (1247489, 150)  Validation set: (155936, 150)  Test set: (155937, 150)


In [5]:
print(pickleData["char_to_int"])

{'|': 90, 'F': 51, 'd': 16, '!': 78, 'V': 67, '?': 48, '{': 94, 'P': 54, 'E': 49, '�': 88, 'B': 46, 'N': 60, 'a': 4, '5': 28, '8': 25, 'l': 15, ']': 97, 'G': 63, '1': 19, 's': 8, 'f': 26, '9': 33, 'M': 58, '0': 20, '补': 0, 'A': 43, 'L': 55, ' ': 89, ';': 57, 'J': 69, '<': 93, '\x82': 87, 'e': 3, 'r': 12, 'g': 24, '#': 84, '7': 34, 'H': 65, 'p': 10, 'Y': 68, 'x': 40, 'Ã': 79, '}': 96, 'X': 72, '-': 18, '`': 92, 'S': 52, 'C': 50, 'b': 21, '4': 31, '&': 44, '2': 22, 'i': 9, 'y': 36, 'T': 59, '_': 42, 'Q': 71, '6': 32, '$': 86, '*': 81, '\x83': 85, '\n': 30, 'O': 62, '~': 75, 'K': 73, '[': 98, 'D': 41, 'z': 47, 'I': 53, '3': 27, '.': 7, 'Z': 70, 'c': 6, 'o': 5, 'W': 66, '\x13': 95, ')': 83, 'h': 14, ':': 29, 'w': 17, 'u': 23, '%': 35, 'Â': 80, '未': 99, ',': 76, 'j': 45, 'v': 38, '=': 39, 'k': 37, '@': 77, '+': 74, 'U': 64, '/': 1, 'R': 61, 'q': 56, '(': 82, "'": 91, 'm': 13, 't': 2, 'n': 11}


In [6]:
print(train_x[2])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0 14  2  2 10 29  1  1  4 13  3 15  9 14  4  9 12  7 12 23  1  9
 13  4 24  3  8  1 32 28 28 20 20 28 31 22 19 27  1 34 31 28 27 28 27 34
 32 33 32 34 33  1]


In [10]:
print(train_y[2])

1


Reconversion of the data into str

In [11]:
#Transform the data back into string using the char_to_int dictionary
def int_to_char(x):
    res=''.join([list(char_to_int.keys())[list(char_to_int.values()).index(i)] for i in x])
    #remove from res the beginning of the string that had zero as a value
    # res=res[res.find('h'):]#is every string starting with 'h'?
    return res

#function that transforms the whole matrix into a list of strings
def matrix_to_list(x):
    res=[]
    for i in range(x.shape[0]):
        res.append(int_to_char(x[i]))
    return res

print(int_to_char(train_x[10]))

补补补补补补补补补补补补补补补补补补补http://www.topky.sk/gl/409791/1859935/Umrela-mu-babka--zacal-patrat-a-nalez-ho-sokoval--Odhalil-nekale-praktiky-okradania-dochodcov


In [18]:
import numpy as np
import tldextract

def feature_vector(x: np.ndarray[str]) -> np.ndarray[int]:
    """
    functions that transform the data into a feature vector, inspired by the article PHISH-SAFE URL Features based Phishing Detection System using Machine Learning.pdf
    feature 1 : number of dots in the URL
    feature 2 : number of hyphens in the URL
    feature 3 : number of @ in the URL
    feature 4 : length of the URL
    feature 5 : number of digits in the URL
    feature 6 : number of // in the URL
    feature 7 : use of "HTTPS"
    feature 8 : number of time "HTTP" appears
    feature 9 : 1 if the IP address is used rather than the domain, 0 otherwise
    feature 10 : detection of suspicious word. equals to the number of the suspicious words, the suspicious word being in the list [token, confirm, security, PayPal, login, signin, bank, account, update]
    feature 11 : Position of Top-Level Domain: This feature checks the position of top-level domain at proper place in URL.

    Have been used in the article but won't be used in our implementation:
    Domains count in URL: Phishing URL may contain more than one domain in URL. Two or more domains is used to redirect address.
    DNS lookup: If the DNS record is not available then the website is phishing. The life of phishing site is very short, therefore; this DNS information may notbe available after some time.
    Inconsistent URL: If the domain name of suspicious web page is not matched  with the WHOIS database record, then the web page is considered as phishing.
    Age of Domain: If the age of website is less than 6 month, then chances of fake web page are more.

    :param x:
    :return a vector of size number of URL * 11 :
    """
    vector = np.array([x.shape[0], 11])
    # handcrafted straight forward features
    for i, url in enumerate(x):
        vector[i][0] = url.count(".")
        vector[i][1] = url.count("-")
        vector[i][2] = url.count("@")
        vector[i][3] = len(url)
        vector[i][4] = sum(c.isdigit() for c in url)
        vector[i][5] = url.count("//")
        vector[i][6] = url.count("https")
        vector[i][7] = url.count("http")
        vector[i][8] = int(tldextract.extract(url).domain == tldextract.extract(url).ipv4)
        # suspicious word detection
        suspicious_words = ["token", "confirm", "security", "PayPal", "login", "signin", "bank", "account", "update"]
        for word in suspicious_words:
            vector[i][9] += url.count(word)
        # top level domain detection: give the rank of the first character of the top level domain in the url
        vector[i][10] = url.find(tldextract.extract(url).suffix)

    # word detection and random word detection
    for url in x:
        tldextract.extract(url)  # https://pypi.org/project/tldextract/
    return vector

In [21]:
feature_vector(int_to_char(np.ndarray([train_x[10]])))

TypeError: only integer scalar arrays can be converted to a scalar index

Data cleansing ?

Statistics ?